# Solution 1D

Nous proposons dans ce notebook de créer une classe adaptée aux solutions d'EDP en dimension 1 d'espace. Cette classe s'appuie sur la classe `mesh1D` afin de prévoir le stockage de la solution approchée en chaque point du maillage.

### Question 1

Complétez la classe `solution1D` : 
* la fonction `init` qui initialise la solution sur le maillage à l'aide de la fonction entrée en paramètre, 
* les fonctions `min` et `max` qui retourne le minimum et le maximum de la solution,
* la fonction d'affichage qui donne des informations sur la solution.

In [1]:
%%file solution1D.hpp

using parameters = std::map<std::string, double>;

/*****************************************************************************************
*
* class solution1D to define the numerical solution in 1D
*
*****************************************************************************************/
class solution1D
{    
public:
    std::vector<double> u; // the numerical solution
    double t;              // the time t 
    mesh1D m;              // the mesh on which the solution is computed
    parameters param;      // the parameters
    
    solution1D(parameters);  // the constructor
    void init(double (*ptr_f)(double)); // initialization of the solution
    
    double max(); // return the maximum of the solution
    double min(); // return the minimum of the solution
};

solution1D::solution1D(parameters p)
{
    param = p;
    // mesh choice (node-centered or cell-centered)
    bool toto = false;
    auto it_nc = param.find("node-centered");
    if (it_nc != param.end())
    {
        if (it_nc->second == 1.)
            m = mesh1D_nc(param);
        else
            m = mesh1D_cc(param);
        toto = true;
    }
    auto it_cc = param.find("cell-centered");
    if (it_cc != param.end())
    {
        if (it_cc->second == 1.)
            m = mesh1D_cc(param);
        else
            m = mesh1D_nc(param);
        toto = true;
    }
    if (!toto)
        throw std::invalid_argument("Missing argument: cell-centered or node-centered mesh!");
    u.resize(m.Nx);
}

void solution1D::init(double (*ptr_f)(double))
{
    // BEGIN SOLUTION
    t = 0.;
    std::transform(m.x.begin(), m.x.end(), u.begin(), ptr_f);
    // END SOLUTION
}

double solution1D::min()
{
    // BEGIN SOLUTION
    return *std::min_element(u.begin(), u.end());
    // END SOLUTION
}

double solution1D::max()
{
    // BEGIN SOLUTION
    return *std::max_element(u.begin(), u.end());
    // END SOLUTION
}

std::ostream& operator<<(std::ostream& out, solution1D& S)
{
    out << std::string(80, '-')    << std::endl;
    out << "solution informations" << std::endl;
    out << std::string(80, '-')    << std::endl;
    // BEGIN SOLUTION
    out << "    dimension:         " << 1      << std::endl;
    out << "    xmin:              " << S.m.xmin << std::endl;
    out << "    xmax:              " << S.m.xmax << std::endl;
    out << "    number of points:  " << S.m.Nx   << std::endl;
    out << "    spatial step:      " << S.m.dx   << std::endl;
    out << "[" << std::endl;
    if (S.m.Nx <= 6)
        for (auto k = 0; k < S.m.Nx; ++k)
            out << "  " << S.m.x[k] << " \t " << S.u[k] << std::endl;
    else
    {
        for (auto k = 0; k < 3; ++k)
            out << "  " << S.m.x[k] << " \t " << S.u[k] << std::endl;
        out << "   ... " << std::endl;
        for (auto k = S.m.Nx - 3; k < S.m.Nx; ++k)
            out << "  " << S.m.x[k] << " \t " << S.u[k] << std::endl;

    }
    out << "]" << std::endl;    
    // END SOLUTION
    out << std::string(80, '-') << std::endl;
    return out;
}

Overwriting solution1D.hpp


### Question 2

Exécutez les cellules suivantes afin d'initialiser la solution à l'aide de la fonction `u0`.

In [2]:
#include <iostream>
#include <vector>

#include "xplot/xfigure.hpp"
#include "xplot/xmarks.hpp"
#include "xplot/xaxes.hpp"

using parameters = std::map<std::string, double>;

#include "mesh1D.hpp"
#include "solution1D.hpp"

In [3]:
// parameters declaration
parameters p = {
    {"xmin", -1},
    {"xmax", 1.},
    {"Nx", 1024},
    {"cell-centered", 1}
};

In [4]:
double u0(double x)
{
    double xmin = p["xmin"], xmax = p["xmax"];
    auto l = 0.75 * xmin + 0.25 * xmax;
    auto r = 0.5 * xmin + 0.5 * xmax;
    if ((x>l) && (x<r))
        return 1.;
    else
        return 0.;
}

In [5]:
solution1D S(p);  // numerical solution
S.init(u0);

In [6]:
xpl::linear_scale xs, ys;
xs.min = p["xmin"], xs.max = p["xmax"];
ys.min = S.min(), ys.max = S.max();
xpl::lines line(xs, ys);
line.x = S.m.x;
line.y = S.u;
line.colors = std::vector<std::string>({"navy"});
xpl::axis hx(xs), hy(ys);
hy.orientation = "vertical";
xpl::figure fig;    
fig.add_mark(line);
fig.add_axis(hx);
fig.add_axis(hy);
fig.title = "discontinuous initial data";
fig.display();

A Jupyter widget

### Question 3

Complétez la fonction `u1` afin d'avoir une fonction globalement continue et polynomiale de degré 2 par morceaux. Vérifiez en exécutant la cellule suivante.

In [7]:
double u1(double x)
{
    // BEGIN SOLUTION
    double xmin = p["xmin"], xmax = p["xmax"];
    auto l = 0.75 * xmin + 0.25 * xmax;
    auto r = 0.5 * xmin + 0.5 * xmax;
    if ((x>l) && (x<r))
        return (x-l)*(r-x)/std::pow(.5*(r-l), 2);
    else
        return 0.;
    // END SOLUTION
}

In [8]:
S.init(u1);
line.y = S.u;
fig.title = "continuous initial data";

### Question 4

Complétez la fonction `u2` afin d'avoir une fonction globalement dérivable et polynomiale de degré 4 par morceaux. Vérifiez en exécutant la cellule suivante.

In [9]:
double u2(double x)
{
    // BEGIN SOLUTION
    double xmin = p["xmin"], xmax = p["xmax"];
    auto l = 0.75 * xmin + 0.25 * xmax;
    auto r = 0.5 * xmin + 0.5 * xmax;
    if ((x>l) && (x<r))
        return std::pow((x-l)*(r-x),2)/std::pow(.5*(r-l), 4);
    else
        return 0.;
    // END SOLUTION
}

In [10]:
S.init(u2);
line.y = S.u;
fig.title = "differentiable initial data";